# Dataset collection

Goal is to collect triplets of data:

- query: `str`
- most relevant pieces of information (ordered): `List[str]`
- answer with a links to a relevant pieces of information: `str`

We are going to use:

- Requests data that we already have (filtering out test, duplicative and no-answer ones)
- Generate new questions over existing data
- Insert new data and generate new question over it
- While inserting, remove very close duplicates

In [1]:
pip install pymongo pandas "thefuzz[speedup]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from tqdm import tqdm
import pandas as pd

pd.set_option('display.max_colwidth', None)

## MongoDB logs

In [3]:
import urllib
from thefuzz import fuzz

import pymongo


mongo_client = pymongo.MongoClient(
    host="ec2-3-77-156-174.eu-central-1.compute.amazonaws.com" + ":" + "27017",
    serverSelectionTimeoutMS=3000,
    username=os.environ["MONGO_INITDB_ROOT_USERNAME"],
    password=urllib.parse.quote_plus(os.environ["MONGO_INITDB_ROOT_PASSWORD"]),
)

DB = mongo_client["backend"]
SOURCE_PATTERN = r"\{ *doc_idx *: *([^}]*)\}"

In [4]:
def drop_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    # create a similarity matrix
    similarity_matrix = []
    for i in tqdm(range(len(df))):
        for j in range(i + 1, len(df)):
            ratio = fuzz.ratio(df.iloc[i]['query'], df.iloc[j]['query'])
            if ratio >= 95:
                similarity_matrix.append((i, j))

    # remove duplicates
    indices_to_remove = set()
    for i, j in similarity_matrix:
        if i not in indices_to_remove and j not in indices_to_remove:
            # dropping first item assuming df was sorted by ascending order
            # so at the end we only left with the lst occurence
            indices_to_remove.add(i)

    df = df.drop(df.index[list(indices_to_remove)])
    return df

In [5]:
requests = pd.DataFrame(DB["requests"].find({}))
requests = requests[
    ["datetime", "query", "model_context", "answer", "api_version", "vendor", "organization", "collections"]
]
requests = requests[requests['api_version'] == 'v2']  # leave results only with links to source docs
requests = requests[
    requests['answer'].str.contains(SOURCE_PATTERN, regex=True)
]  # remove answers w/o sources (means that no answer was found)
requests = requests[
    ~requests['answer'].str.contains('|'.join(['I was unable', 'hello', 'unfortunately']), case=False, na=False)
]  # info not found but model still provides false link
# requests = requests.drop_duplicates("query", keep="last")
requests = drop_duplicates(requests)  # dropping based on levenshtein distance

/tmp/ipykernel_261197/1126407693.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  requests['answer'].str.contains(SOURCE_PATTERN, regex=True)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 499/499 [00:07<00:00, 65.26it/s]


In [6]:
requests

datetime  \
12   2023-07-03 12:37:11.945   
20   2023-07-03 13:54:12.036   
21   2023-07-03 13:54:23.358   
25   2023-07-03 14:02:12.462   
26   2023-07-03 14:02:47.150   
...                      ...   
2540 2023-08-10 00:41:01.597   
2542 2023-08-10 09:17:35.233   
2544 2023-08-10 13:09:50.533   
2545 2023-08-10 13:53:55.329   
2548 2023-08-11 10:29:36.409   

                                                                   query  \
12                                                             password?   
20                                  Why ai will not substitute humanity?   
21                                                      how to  get sim?   
25                                 Can we install antivirus on primebook   
26                                           Will AI supersede humanity?   
...                                                                  ...   
2540  im looking for a self adhesive approved sticker for industrial use   
2542                                         how to troubeshoot for RSSI   
2544                                    how to configure screen off time   
2545          i click to word article but i redirected to sso login page   
2548                                                           primebook   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  